# Hoare Logic

Method by `Tony Hoare` to verify that an imperative program is correct with respect to a given specification, i.e., reasoning about program correctness using _pre_ and _post_ conditions.  

## Summary

-   **Goal**: to prove that a program `S` is correct with respect to a
    given precondition `P` and postcondition `Q`.
-   **Method**: proving Hoare tripple
    -   Computing **weakest precondition** of `S` with respect to `Q`,
        i.e., `wp(S, Q)`
    -   If for every loop in `S`, need to provide a **loop invariant**
    -   Form **verification condition** (**vc**): `P` `\implies`
        `wp(S, Q)`
        -   If this vc is true, then the Hoare tripple is valid, i.e.,
            `S` is correct wrt to `P` and `Q`
        -   Else, Hoare tripple is invalid but we cannot conclude that
            `S` is incorrect wrt to `P` and `Q` (it could be that we
            picked a weak loop invariant)


## Hoare Tripple

- Hoare Tripple  {`P`} `S` {`Q`}:
    - `S`: a program (a list of statements)
    - `P`: precondition (a formula representing the _precondition_ that hold before the execution of `S`)
    - `Q`: postcondition (a formula representing the _postcondition_ that holds after the execution of `S`)
    - Read:  assume `P` holds, if `S` executes successfully (i.e., terminates), then `Q` holds
      - E.g., {`x = 5 & y > 2`} `z := x + y; z := z + 2` {`z > 9`}:   assume `x=5 and y > 2`, if `z:x +y; z := z + 2` runs successfully, then we have `z > 9`
    - Meaning: if the tripple is _valid_, then `S` is correct wrt to the program specification (`P` and `Q`)
    
Notation: things inside {} are treated as formulae, so `P` and `Q` are formulae; `:=` is an assignment operator.

### Partial and Total Correctness

**Partial**: assume `P` holds, if `S` *successfully* executes, then `Q` holds
-   Here, we **assume** the program terminates (`S` successfully executed)

**Total**: assume `P` holds, `S` *successfully* executes and `Q` holds
-   Here, we **require** `S` terminates
-   Dififcult because have to ensure the termination of `S`

We often just use partial correctness because total correctness requires us to also prove that `S` would terminate, which is a difficult problem (theoretically undecidable).


### Examples of Hoare Tripples

1. `{True} x := 5 {x = 5}`:  *valid* 
2. `{True} x := 5 {x=5 or x= 6 or x > 6}`:  *valid*
3. `{True} x := 5 {x > 1}`:  *valid*
4. `{True} x := 5 {x > 6}`:  *invalid* ,  because after executing `x:=5` , we cannot have `x > 6`

*Strongest* postcondition: The postcondition in **1.** is strongest (i.e., `x=5` is more precise than `x > 1` or `(x=5 or x=6 or x > 6)`.  In general we want the strongest (most precise) postcondition.

5. `{x = 1 & y = 2} z:= x/y  {z < 1}`:  *valid* 
6. `{x = 2 & y = 4} z:= x/y {z <1}`: *valid*
7. `{0 < x < y & y != 0} z:= x/y {z <1}`: *valid*   

*Weakest* precondition: the precondition in **7.** is weakest (i.e., it is the least constraint precondition).


8. `{x < y} z:= x/y  {z < 1}`:  *Invalid*, counterexample input x=-1, y=0: after executing z:=x/y, we do not have z < 1 (we got a div-by-0 exception)
9. `{x = 0} z:= x/y  {z < 1}`:   *Invalid*, counterexample input x=0, y=0 
10. `{y != 0} z:= x/y  {z < 1}`:  *Invalid*, counterexample input x=2 , y=1 
11. `{x < y & y != 0} z:= x/y {z <1}`: *Invalid*, counterexample input   x=-2,  y=-1


### In-class Questions

1. `{False} x:=3 {x = 8}`:  *Valid*, because the `False` precondition, i.e., there is no way we can satisfies the precondition in the first place.
   ```
2. `{???} x:= y - 3 {x = 8}`
    A: `{y = 11}`

3. `{x = y} ??? {x = y}`
   A: `skip`

4. `{x < 0} while(x!=0) do x:=x - 1 {???}`
   A: any formula, because the loop *does not* terminates so the Hoare tripple is valid no matter what postcondition we use.
   Note: this tripple is valid for *partial* correctness, but is *not* valid for *total* correctness.

# Program Verification using Hoare Logic

We can automatically verify (partial) program correctness using Hoare Triples and weakest preconditions.
To prove `{P} S {Q}` is valid, i.e., to prove the program `S` is correct wrt to the precondition `P` and postcondition `Q`, we check `P => wp(S, Q)`, where the function *wp* returns the weakest precondition allowing the program `S` to achieve the postcondition `Q`.
  - Note: `=>` is the logical implication operator
  
Thus, to show the validity of `{P} S {Q}`, we show that `P` implies (`=>`) the *wp* of `S` wrt to `Q`. Hoare defines rules to obtain the WP of different kind of (imperative) program statements as shown below.



## WP's for statement `S` im imperative program

| Statement Type | `S`                    | `wp(S, Q)`                                                       | Comment                                                                  |
|----------------|:----------------------:|------------------------------------------------------------------|--------------------------------------------------------------------------|
| Assignment     | `x := E`               | `Q[x/E]`                                                         | replace all occurences of the variable `x` in `Q` with the expresion `E` |
| Skip           | `skip`                 | `Q`                                                              | skip statement: no effect                                                |
| Sequential     | `S1;S2`                | `wp(S1, wp(S2,Q))`                                               |                                                                          |
| Conditional    | `if b then S1 else S2` | `b => wp(S1,Q) & \neg b \Rightarrow wp(S2,Q)`                    |                                                                          |
| Loop           | `while b do S`         | `(I) & (I & B \Rightarrow wp(S,I)) & (I & \neg B \Rightarrow Q)` | `I` is a user supplied _loop invariant_                                  |



### S is an **ASSIGNMENT** statement

    WP(x := E, Q) = Q[x/E]

- That is, we substitute all occurences of `x` in **Q** with the expression `E`

- Example

```
WP(x:=3, x + y = 10)
 = 3 + y = 10 
 = y = 7  # Thus, we have {y=7} x := 3 {x + y = 10}

WP(x:=3, {x + y > 0) 
  = 3 + y > 0
  = y > -3  # Thus, we have {y > -3}  x := 3 {x + y> 0}
```

### S is a **LIST of Statements**

    WP([S1; S2; S3 ...;], Q) = WP(S1, WP(S2;S3;.., Q))
    WP([], Q) = Q

-   Example

        WP([x:=x+1; y:=y*x], y=2*z) 
            =   WP(x:=x+1, WP([y:=y*x], y=2*z))
            =   WP(x:=x+1, y*x=2*z)
                =   y*(x+1)=2*z # Thus, we have {yx+y=2*z}  x:=x+1; y:=y*x {y=2*z}


### S is a **CONDITIONAL** statement

    WP(if b then S1 else S2, Q)  =  (b=> WP(S1,Q))  &  (!b => WP(S2, Q))

-   Example


        WP(if x > 0 then y := x + 2  else y:= y + 1,  y > x) 
        = (x>0 => WP(y:=y+x, y>x) & (x<=0 => WP(y:=y+1, y>x))
        = (x>0 => y+x>x)  &  (x <= 0  =>  y+1>x)
        = x>0 => y>0  & x<=0 => y+1>x


        WP(if x > 0 then y :=x  else y:= 0,  y > 0) 
        = (x>0 => WP(y:=x, {y >0})) & (x<=0 => WP(y:=0, y>0))
        = (x > 0 => x > 0)  &  (x <= 0  =>  0 > 0)
        = True & x > 0   
        = x > 0 # (0> 0 is false,  and so !(x<=0) or false is !(x<=0) = x>0)
        This WP ~x > 0~ shows that we can only use the first branch and discard the second branch (which always violate the postcondition).

-   Instead of using `=>` (imply), which might be confusing to some,
    we can use `!` (not) and `||` (or)

    -   WP(if b then S1 else S2, Q) = (b=\> WP(S1,Q)) & (!b =\>
        WP(S2, Q)) = !((b & !WP(S1,Q)) \|\| (!b & !WP(S2, Q)))



### S is a **LOOP**

-   *Important*: unlike other statements where we can compute WP
    automatically, for loop, we (the user) need to supply a *loop
    invariant* to obtain the WP of loop

-   **Loop invariant** `I`: captures the meaning of the loop
    (manually provided by you)

    -   property that holds when the loop entered

    -   is preserved after the loop body is executed (inductive loop
        invariant)

    -   Where is the loop invariant **I**?

        -   loop entrance (note: does not mean the location AFTER
            you enter the loop)

        -   This transformation can help you visualize **I**:

                while (b){
                    //loop body
                }

        this loop is equivalent to

            while (True){
                [I]      // loop invariant I is right here
                if (!b) break
                //loop body
            }

            WP(while [I] b do S, Q) =  I & (I & b => WP(S,I) & (I & !b)  => Q)

-   where `I` is a loop invariant

-   the wp is a conjunction
    `I & (I & b => WP(S,I) & (I & !b)  => Q)`, which consists of 3
    conjuncts:

    1.  `I` : the loop invariant (should hold when entering the
        loop)
    2.  `(I & b)  =>   I` : (entering the loop because `b` is true)
        `I` is preserved after each loop body execution
    3.  `(I & !b) =>  Q` (exiting the loop because `b` is false),
        when exiting the loop, the post condition holds

    -   Example

            {N >= 0}   // precondition

            i := 0 ;

                while(i < N)
                i := N;

                {i = N} // post condition

-   many possible loop invariants

        - i <= N
        - i>=0 
        - N >= 0
        - TRUE  (very weak and trivial)

    -   but need to use a loop invariant that is sufficiently
        strong, otherwise we cannot prove the program (so unless the
        postcondition we want to prove is very weak (e.g., `TRUE`),
        then unlikely we want to use `TRUE` as a loop invariant, even though TRUE is loop invariant).


### Example 1 (*sufficiently* strong invariant): 

attempt to use the loop invariant `i <= N` to prove the program

    WP([i := 0; while[i<=N] i < N do i:= N], i = N) 
    = WP(i := 0; WP(while[i<=N] i < N do i:=N], i = N) 
    = 
    // Let's first compute WP(while[i<=N] i < N do i:=N, {i = N}). According to the WP rule for LOOP, we will have 3 conjuncts 
        1. i <= N

        2. (i <= N & i < N) => WP(i:=N, {i<=N})
            = i < N          =>  N <= N 
            = i < N          =>  True   
            = True     // because !(i<N) | True  is True (anything or with True is True)

        3. (i <= N & !(i<N)) => i = N
            = i = N           => i = N
            = True     // because !(i=N) | i = n  is True (a or !a is True)

        =  i <= N & True & True
        =  i <= N

    //after obtaining the WP i<=N for the while loop, we continue with the statement i:=0
    // WP([i := 0; while[i<=N] i < N do i:= N], i = N)  = WP(i := 0, i<=N)
    WP(i := 0, i<=N)
    = 0<=N  //we obtain this by applying the WP rule for assignment

    //now we construct a "verification condition" (vc) to check that the given precondition implies the WP 0<=N
    P => WP([i := 0; while[i<=N] i < N do i:= N], {i = N}) 
    = N>=0 =>  0<=N   // N>=0 is the given precondition and 0 <= N is the WP we obtain above for the program
    = True
    //the given precondition N>=0 implies 0<=N, and thus the the Hoare tripple is valid, i.e., the program is correct 
    //Thus using this loop invariant i<=N we can prove the validity of the Hoare tripple.
    // The loop invariant i <= N is *sufficiently strong* to let us prove the program satisfy the specifications.


### Example 2 (*insufficiently* strong): 

attempt to use the loop invariant `N >= 0` to prove the program

    WP(while[N >= 0] i < N do i:=N, {i = N})
    =
        1. N >= 0
        2. (N >=0 & i < N) => WP(i := N, N >= 0) = 
            (N >=0 & i < N) => i >= 0   // we can't simplify much, so just leave as is

        3. N >=0 & !(i<N) => i =N
            (N >= 0 & i >= N) => i = N
            i>= 0  => i = N  // we can't simplify much, so just leave as is

        =  N >=0 & (N >=0 & i < N) => i >= 0 & (i>= 0  => i = N)

    WP(i:=0; {N >=0 & (N >=0 & i < N) => i >= 0 & (i>= 0  => i = N)})
        = (0 >= 0) & (0 >= 0 & 0 < N => 0 >= 0) & (0>=0 => 0 = N)  //apply WP for assignment and simplify
        =  TRUE    &      TRUE                  & 0 = N
        = 0 = N

    // The vc is then
        P => 0 = N  // the given precondition implies 0 = N
        (N >= 0) => 0 = N   (Not simplified to TRUE, i.e., invalid Hoare tripple)
        // Thus using this loop invariant is not sufficiently strong as we cannot use it prove the validity of the Hoare tripple.
        // *Important*: not being able to prove simply means we cannot prove it using this loop invariant.  It *does not* mean that you disprove it or show that the Hoare triple is invalid.  (in fact, we know the Hoare tripple is valid if we used a different loop invariant, e.g., i <= N)


### WP for While Statement

Notice the weakest precondition for while statement
`(I) & (I & B \Rightarrow wp(S,I)) & (I & \neg B \Rightarrow Q)`
relies on a user-supplied loop invariant `I`.
Indeed, while we can automatically computes the wp's for other statements automatically, we need the user to provide us a loop invariant to compute the wp's for while loop. 


A loop invariant is a property that (i) holds upon the entrance of the loop and (ii) preserves after the execution of the loop body.
In essense, the loop invariant captures the semantics of the loop and thus allows us to replace the iteration details of the loop with the invariant.
Often, the loop invariant properties are expressed as logical formulas.

In the below example 
```c
i := 0;
while (i < N) do
  i := N;
```
`N \ge 0` and `i \le N` are valid loop invariants. Note, thus `N \ge 0 & i \le N` is also a loop invariant.  In contrast,  `i= 0` and `i = N` are not loop invariants.
In general, there are many possible loop invariants, e.g., `\text{True}, i \le N + 1, \dots` in this example.  




## Sufficiently Strong Loop Invariants

Like postconditions, we generally want _strong_ loop invariants to represent the more precise semantics of the program. 
However, just like postconditions, determining the strongest (or even strong) loop invariants can be nontrivial, especially when we have to manually come up with them.
Fortunately, in many cass, we do not need to have strong loop invariants, but just _sufficiently_ strong to prove program specifications.
For example, if we want to prove a weak postcondition, then we only need sufficiently strong invariant to establish that proof.




## Example

We show that the _multiplication_ program below is correct with respect to the precondition `P = \{A \ge 0 & B \ge 0 \}` and `Q=\{z = A*B\}`.
We will use the loop invariant `I = \{A*B = z + x*y\}` to demonstrate this.

```C
// P = {A >= 0 && B >= 0}
x := A; 
y := B; 
z := 0; 
// L1
while x > 0 do
    if odd(x) then 
        z := z + y
    y := 2 ∗ y; 
    x := x / 2;   //integer division
    // L2

// L3: Q = {z == A*B}
```


1. Initial (at L1, when we about to enter the loop): 
    - `(A \ge 0 & B \ge 0) & (x = A & y = B & z = 0) \Rightarrow (A*B = x + y)`.  Checked!

- Terminate (at L3, when the loop exits):
    - `(A*B = z + x*y & x = 0) \Rightarrow (A*B = z)`. Checked!
    - Note, the loop exit is `x \le 0`, however we know x cannot be negative so we strengthen it to `x 
= 0`

- Loop (at L2, the loop inv is preseved at the end of the loop body): we show that `(I & x > 0) \Rightarrow wp(\text{loop_body}, I)`.  In this example, the loop body consists of 2 cases:
   - Case 1: when x is even:
       - `x > 0 & A*B = z + x *y \Rightarrow ((A*B = z + x*y)_y^{2*y})_x^{x/2}` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = z + (x/2)*2*y)` \
       = `x > 0 & A*B = z + x *y \Rightarrow A*B = z + x*y`.  Checked

   - Case 2: when x is odd, then due to intger division, we have `x = 2*u + 1`, e.g., if `x = 7`, then `u = 3`:
       - `x > 0 & A*B = z + x *y \Rightarrow (((A*B = z + x*y)_y^{2*y})_x^{x/2})_z^{z+y}` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = (z+y) + (x/2)*2*y)` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = (z+y) + ((2u + 1)/2)*2*y)` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = (z+y) + (2u/2 + 1/2)*2*y)` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = z + y + u * 2*y)` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = z + y * (1 + u * 2))` \
       = `x > 0 & A*B = z + x *y \Rightarrow (A*B = z + y * x)`.  Checked
       
We also note that this program terminates because (i) if `x \le 0`, we skip the loop and (ii) otherwise, ie.., `x > 0`, we enter the loop but at each iteration we have `x / 2` and thus eventually this integer division will result in `x = 0` and we exit the loop.



## Example

Given the below program with the precondition `P = \{x \le 9\}` and the postcondition `Q = \{x = 12\}`
```C
    x := x + 1;
    while x != 10 do
       x := x + 1;
    x := x + 2
```

1. Find an invariant `I` for the loop (that hopefully is sufficiently strong to prove `Q`)
   - Guess `x \le 10`

- Find the *wp* of the loop
   - wp(x := x + 1; while x != 10 do x := x + 1; x := x + 2, `x = 12`) \
    = wp(x := x + 1; while x != 10 do x := x + 1, `x = 10`) \
    = wp(x := x + 1, wp(while x != 10 do x := x + 1, `x = 10`) \
    = wp(x := x + 1, `(x\le 10) & (x \le 10 & x \ne 10 \Rightarrow wp(\text{x := x + 1}, x \le 10)) & (x \le 10 & x = 10 \Rightarrow x = 10)`) \
    = wp(x := x + 1, `x\le 10 & \text{True} & \text{True}`
    = `x \le 9`

- Prove the program is correct
   - `P \Rightarrow`  wp(x:= x + 1; while x != 10 do x := x + 1; x := x + 2, `x = 10`) \
   = `x \le 9 \Rightarrow x\le 9` \
   = True
- Argue the program terminates
  - The postcondition states that `x \le 10`.  If `x = 10`, then we skips the loop and terminates the program.  Otherwise (`x < 10`), `x` is incremented by 1 at each iteration and thus eventually we will get `x == 10`, exit the loop, and terminate.



## Example

Given the below program with the precondition `P=\{ N \le 10 \}`
```c
i := 0;
while (i < N) do
  i := N
```

1. Determine what this program does and write a post condition `Q` for it
  - `Q = \{ i = N \}`
  
- Find an invariant `I` for the loop that is sufficiently strong to prove `Q`
  - `i \le N`.  Note: something like `N \le 0` is not sufficient
  
- Prove that this program is correct with respect to `P` and `Q`
    1. First, we find the wp of the program (we would use `I` for the loop) \ 
     wp(i := 0; while i < 10 do i := N, `i \le N`) \
       = wp(i := 0, wp(while i < 10 do i := N, `i \le N`))\
       = wp(i := 0, `(i \le N & i < 10 \Rightarrow i \le N) & (i \le N & i \ge N \Rightarrow i = N)`)\
       = `(0 \le N & 0 < 10 \Rightarrow 0 \le N) & (0 \le N & 0 \ge N \Rightarrow 0 = N)`

    - Then, we show that `P \Rightarrow wp`\
    `N \le 10 \Rightarrow (0 \le N & 0 < 10 \Rightarrow 0 \le N) & (0 \le N & 0 \ge N \Rightarrow 0 = N) = \text{True}` \
    - In fact, note that the wp of the loop can be simplified to just True and thus it does not matter what the precondition or the assignment statement before the loop is.

- Argue that this program terminates

In [ ]:


# unused

- {`x = y`} x := x + 3 {`x = y+3`}
- {x `>` -1} x:=2*x + 3 {`x > 1`}
- {`x = a`} if x `<` 0 then x := -x {`x = |a|`}

## Strongest Postconditions

Consider the Hoare tripples below:
1. {`x = 5`} x:=x*2 {`true`}
1. {`x = 5`} x:=x*2 {`x > 0`}
1. {`x = 5`} x:=x*2 {`x = 10 \vee = 5`}
1. {`x = 5`} x:=x*2 {`x = 10`}

All of these tripples are valid, but the last one with the postcondition  `x =` 10 is the *strongest*, i.e., most precise.  In general, we want _strong postconditions_ for our specifications.  However, there could be cases when we do not need the strongest postconditions, e.g., if we only want to show that x remains positive, then the postcondition `x > 0` would suffice.

### Definition
- In {P} S {Q}, Q is the strongest postcondition if `\forall Q'. \{P\} S \{Q'\} & Q \Rightarrow Q'`. This says that Q is the strongest postcondition if (i) it is a postcondition in the tripple {P} S {Q} and it is stronger than any other postconditions Q' of the tripple {P} S {Q'}.

For the example above, x `=` 10 is the _strongest_ postcondition because
  - `x = 10 \Rightarrow true`
  - `x = 10 \Rightarrow x>0`
  - `x = 10 \Rightarrow (x= 10 \vee x= 5)`
  - `x = 10 \Rightarrow x= 10`.
  - `x = 10` is also stronger than any other postconditions of this example (e.g., `x > 1,  x  = 10 \vee x > 3, \dots`). 

Note that if **False** is a postcondition in the tripple {P} S {Q}, then it would be the strongest possible postcondition because False implies anything.


## Weakest Preconditions

Consider the Hoare tripples below:
1. {`x = 5 & y = 10`} z := x/y {`z<1`}
- {`x<y & y>0`} z := x/y {`z<1`}
- {`y\neq 0 & x/y <1` } z := x/y {`z<1`}

All of these triples are true, but the last one `y\neq 0 & x/y <1` is the most general or _weakest_ precondition.  In general, we want  _weak preconditions_ for our specifications because it allows us to run the programs with fewer assumptions or restrictions.

### Definition
- In {P} S {Q}, P is the weakest precondition if 
`\forall P'. \{P'\} S \{Q \} & P' \Rightarrow P`.

  